##  Preprocessing of raw images 
 
We chose natural images from the __Hateren Schaaf database__ originally used [here](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1688904/)

We selected a subset of 137 images that did not contain any human created forms and were not blurred. They were originally preselected in this [paper](https://pubmed.ncbi.nlm.nih.gov/16782164/)

Here we perform preprocessing of the images that then will be fed to the generative algorithms. The images are blocked averaged, scaled and shifted and then passed through a Gaussian filter which aims to emulate retinal ganglion cells. 

After following the directions below on where and how to download the images, you run sequentially all the cells below to process and store the data so that the produced image patches can be used by the generative algorithms.

#### Download the __Hateren Schaaf database__ images

From the website http://bethgelab.org/datasets/vanhateren/ download the *.IMC files, unzip and store them in the directory __sparseRegularizers/databaseHaterenSchaaf/vanhateren_imc/__

````
mkdir -p databaseHaterenSchaaf
cd databaseHaterenSchaaf
wget http://pirsquared.org/research/vhatdb/full/vanhateren_imc.zip
mkdir -p vanhateren_imc
cd vanhateren_imc
unzip vanhateren_imc.zip
rm vanhateren_imc.zip
```

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import array
import os

import utils
from generativeModels import SparseModel, runModelSim

### Functions

In [11]:
def getImages(filenamesSelect, baseAddress='', height=1024, width=1536):
    
    imgAll = np.zeros((height,width,len(filenamesSelect)))
    for ind,filename in enumerate(filenamesSelect):
    
        fileLoc = baseAddress+filename
        with open(fileLoc, 'rb') as handle:
           s = handle.read()
        arr = array.array('H', s)
        arr.byteswap()
        imgAll[:, :, ind] = np.array(arr, dtype='float').reshape(height, width)

    return imgAll

In [3]:
#we select the min and the max to rescale from each separate image (instead of taking the max and min of all)
#the pixels become from zero to one
#then we take the mean of the pixels of each image and subtract each pixel of the image with that value
#the mean of the pixels for each image become zero
def rescaleMeanCenterImgs(blockImgAll):

    maxVals = np.max(np.reshape(blockImgAll,(blockImgAll.shape[0]*blockImgAll.shape[1],-1)),axis=0)
    minVals = np.min(np.reshape(blockImgAll,(blockImgAll.shape[0]*blockImgAll.shape[1],-1)),axis=0)
    
    normImgAll = (blockImgAll-minVals)/(maxVals-minVals)

    meanVals = np.mean(np.reshape(normImgAll,(normImgAll.shape[0]*normImgAll.shape[1],-1)),axis=0)
    stdVals = np.std(np.reshape(normImgAll,(normImgAll.shape[0]*normImgAll.shape[1],-1)),axis=0)
    imgAllScaleShift = (normImgAll - meanVals)/stdVals
    
    return imgAllScaleShift

In [4]:
#whitens the images M images of heightXwidth blockImgAll[height,width,M] with a difference of Gaussian filter
#representing the response properties of retinal ganglion cells. Subsequently normalize the images so that they
#all have the same variance (0.1).
def whitenDoG(blockImgAll):

    height,width,M = blockImgAll.shape
    print(f'There are {M} images, each of height {height} and width {width}')
    heightHalf = int(height/2); widthHalf = int(width/2)
    print(f'half of the height is {heightHalf} and half of the width is {widthHalf}')
    heightInd = np.arange(-heightHalf,heightHalf)
    widthInd = np.arange(-widthHalf,widthHalf)
    fx,fy = np.meshgrid(heightInd,widthInd, indexing='ij')
    rho = np.sqrt(fx*fx + fy*fy)
    f0 = 0.4*height
    exponent = -(rho/f0)**4
    filt = rho*np.exp(exponent)

    imgAllFilt = np.zeros_like(blockImgAll)
    for i in np.arange(M):
        imF = np.fft.fft2(blockImgAll[:,:,i])    
        imgAllFilt[:,:,i] = np.fft.ifft2(imF*np.fft.fftshift(filt)).real

    normFactor = np.sqrt(np.mean((np.std(np.reshape(imgAllFilt,(imgAllFilt.shape[0]*imgAllFilt.shape[1],-1)),axis=0)**2)))
    var = np.sqrt(0.1)
    imgAllFiltNorm = var*imgAllFilt/normFactor

    return imgAllFiltNorm


In [5]:
#createRandomCropImagesAndRs gets randomly selected cropped image patches 
#the sequences can be replicated and used for comparisons since we use a random seed
#numCrImages is the same as numRs and it is numIter*batch_size
#sz: size of cropped image szXsz   
#numImages: number of images from where we take the cropped images
#imgs: the images from where we take the cropped images

def createRandomCropImages(numCrImages,sz,imgs, seed = 0):


    np.random.seed(seed)
    H,W,numImages = imgs.shape
    
    #GET THE IMAGES FIRST
    #where to start the upper left corner of the cropped image
    beginx = np.random.randint(0, W-sz, numCrImages)
    beginy = np.random.randint(0, H-sz, numCrImages)
    #from which image to crop
    indImgs = np.random.randint(0, numImages,numCrImages)
    inputs_list = []

    # Get cropped images, normalize them, and store them in inputs_list
    for i in range(numCrImages):        
        img = imgs[:, :, indImgs[i]]
        clop = img[beginy[i]:beginy[i]+sz, beginx[i]:beginx[i]+sz].flatten()
        inputs_list.append(clop - np.mean(clop))

    inputsAll = np.array(inputs_list)

    return inputsAll

#### Get the images with the indeces below preprocess them, get the input image patches and save them

In [6]:

# indices of the 137 preselected images
imagesNum = ['0061','0094','0199','0225','0353','0369','0388','0472','0477','0538','0619','0699','0764','0791','0827','0887'
,'0960','1024','1041','1067','1102','1116','1142','1150','1186','1204','1274','1288','1377','1385','1692','1723'
,'1749','1778','1829','1848','1867','1875','1893','1899','1902','1966','1993','1996','2066','2098','2118','2131'
,'2162','2198','2205','2261','2280','2294','2310','2341','2350','2369','2397','2405','2473','2508','2511','2516'
,'2535','2537','2562','2569','2573','2597','2615','2637','2671','2715','2796','2822','2895','2996','3004','3014'
,'3038','3053','3104','3116','3178','3181','3198','3224','3240','3251','3268','3305','3335','3362','3371','3388'
,'3420','3421','3432','3437','3451','3491','3509','3527','3532','3587','3613','3660','3664','3671','3741','3748'
,'3762','3802','3808','3831','3835','3848','3896','3929','3969','3998','4000','4027','4029','4049','4053','4080'
,'4089','4101','4103','4115','4153','4180','4184','4202','4204']

filenamesSelect = []
for part in imagesNum:
    whole = 'imk0'+part+'.imc'
    filenamesSelect.append(whole)


baseAddress = 'databaseHaterenSchaaf/vanhateren_imc/'

#Get original images
imgAll = getImages(filenamesSelect,baseAddress)
fileSaveRaw = 'data/imagesField.pckl'
utils.saveVar((filenamesSelect,imgAll),fileSaveRaw)

#rescale its image to be between zero and one
preproImgAll = rescaleMeanCenterImgs(imgAll)

#whiten the images as in:  
#Olshausen BA, Field DJ (1997)  Sparse Coding with an Overcomplete  Basis Set: A Strategy Employed by V1?  Vision Research, 37: 3311-3325.
imgAllFiltNorm = whitenDoG(preproImgAll)
fileSave = 'data/DoGImagesScaleShift.pckl'
utils.saveVar(imgAllFiltNorm,fileSave)


numIter = 4000 ;batchSize = 250
numCrImages = numIter*batchSize
sz = 16 #size of one side of the square patch
seed = 0
print(f'The algorithm iterates a batch of {batchSize} cropped images of size {sz}X{sz} {numIter} times')
print(f'So in total we use {numCrImages} patches')

#Get the cropped imagess that we are going to use for all algorithms
inputsAll = createRandomCropImages(numCrImages,sz,imgAllFiltNorm, seed)
print(f'the number of patches is {inputsAll.shape[0]}(0 dimension) and the size of each patch is {inputsAll.shape[1]}(1 dimension)')

fileSave = 'data/DoGPatchesScaleShift.pckl'
utils.saveVar(inputsAll,fileSave)


FileNotFoundError: [Errno 2] No such file or directory: 'databaseHaterenSchaaf/vanhateren_imc/imk03741.imc'

### Load and show examples of raw and whitened images

In [ ]:
#Load raw and whitened images
fileLoad = 'data/imagesField.pckl'
(filenamesSelect,imgsRaw) = utils.loadVar(fileLoad)

fileLoad = 'data/DoGImagesScaleShift.pckl'
imgsWhitened = utils.loadVar(fileLoad)

# Plot datasets
numImages = 10
fig, axs = plt.figure(numImages, 2, figsize=(15, 15*numImages/2))
for i in range(numImages):
    axs[0][i].imshow(imgsRaw[:, :, i], cmap="gray")
    axs[1][i].imshow(imgsWhitened[:, :, i], cmap="gray")
    plt.axis("off")
    
    
plt.tight_layout()
fig.suptitle("First Row: Raw Images | Second Row: Whitened Images", fontsize=40)
plt.subplots_adjust(top=0.9) 
 

## convert to PNG

In [30]:
height = 1024
width = 1536

import glob
import cv2

for filename in glob.glob('databaseHaterenSchaaf/vanhateren_imc/*.imc'):
    # print(filename)

    with open(filename, 'rb') as handle:
        s = handle.read()
        arr = array.array('H', s)
        arr.byteswap()
        image = np.array(arr, dtype='float').reshape(height, width)

    # imageio.imsave(filename.replace('vanhateren_imc', 'vanhateren_imc_png').replace('.imc', '.png'), image)
    # print('>', image.min(), image.max())
    n_bits = 16
    image /= (image.max()-image.min())
    # print('<', image.min(), image.max())
    image *= 2**n_bits-1 # in the uint range
    image = image.astype(np.uint16)
    cv2.imwrite(filename.replace('vanhateren_imc', 'vanhateren_imc_png').replace('.imc', '.png'), image)


> 0.0 31740.0
< 0.0 1.0
> 9.0 6237.0
< 0.001445086705202312 1.0014450867052023
> 0.0 7348.0
< 0.0 1.0
> 1.0 10711.0
< 9.337068160597572e-05 1.000093370681606
> 0.0 29919.0
< 0.0 1.0
> 0.0 18161.0
< 0.0 1.0
> 4.0 8081.0
< 0.0004952333787297264 1.0004952333787298
> 2.0 8071.0
< 0.00024786218862312553 1.000247862188623
> 0.0 12506.0
< 0.0 1.0
> 99.0 8623.0
< 0.011614265603003285 1.0116142656030034
> 43.0 14690.0
< 0.0029357547620673177 1.0029357547620674
> 1.0 14360.0
< 6.964273278083432e-05 1.0000696427327809
> 99.0 2831.0
< 0.03623718887262079 1.0362371888726207
> 1.0 8470.0
< 0.0001180776951233912 1.0001180776951233
> 66.0 7377.0
< 0.009027492819039803 1.0090274928190397
> 72.0 4827.0
< 0.015141955835962145 1.0151419558359622
> 40.0 6526.0
< 0.006167129201356768 1.0061671292013568
> 51.0 8275.0
< 0.00620136186770428 1.0062013618677044
> 14.0 21907.0
< 0.0006394738044123693 1.0006394738044124
> 0.0 6148.0
< 0.0 1.0
> 0.0 4732.0
< 0.0 1.0
> 0.0 8325.0
< 0.0 1.0
> 71.0 7299.0
< 0.00982291